In [1]:

import transformers
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ast
import os
import re

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.model_selection import train_test_split


from sklearn import metrics
from tqdm import tqdm
from torch.utils.data import random_split
import datasets



c:\Users\acer\miniconda3\envs\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'datasets'

In [2]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
device

'cuda'

In [3]:
device = 'cuda:1'

In [4]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"


# **Importing and Pre-Processing the domain data**

In [5]:
all_data = pd.read_csv('origin\\all_dedup.csv')

In [6]:
all_data

,id,text,label,dataset_type
0,0,Bấp bênh vl thế,1,train
1,1,Chắc cũng biết ko tồn tại đc bao lâu nữa nên c...,1,train
2,2,Thấy chán ad page này kiến thức thì nông cản c...,1,train
3,3,Giang Giang Đỗ Thị Ngọc Hà trend mới kìa kìa,0,train
4,4,đcm 😒 sau có con cho hút cỏ chữa bệnh chứ đéo ...,1,train
...,...,...,...,...
46002,46002,"trong trường hợp này, theo ls. khanh, bà hằng...",1,test
46003,46003,bà cũng không ít lần chỉ trích nặng nề những ...,0,test
46004,46004,“bà nguyễn phương hằng có nhiều biểu hiện đã ...,0,test
46005,46005,vũ đức khanh nói. bà nguyễn phương hằng bắt đ...,0,test


In [53]:
data.rename(columns={'comments':'text'}, inplace=True)

In [54]:
data

,index,text,label
0,0,Giờ biết giải thích sao với lão tôn đây 😅,1
1,1,Nguyễn Đức Duẩn Thanh niên nhà Hà Nội thích N ...,0
2,2,Khỉ bố đang rất đau khổ,0
3,3,Hàng xóm bên cạnh ngta cũng phận nữ nhi mà các...,1
4,4,"Sinh nở đã vất vả lắm rồi, đừng hỏi bố đứa trẻ...",1
...,...,...,...
6693,6693,"Khổ nhưng vẫn đc độc lập tự do,được đón lễ lớn...",0
6694,6694,30/4 3 que đu càng tụt quần bay trong gió😂,0
6695,6695,ba que khát nước,0
6696,6696,Hoan hô Việt Tân đưa tin rất hay và thời sự,0


# **Infer**

In [55]:
from infer import Model


In [56]:
device = 'cuda:1'
model = Model(weight= 'ckpt/model-fine-tune-final.pth', device= device)

In [57]:
toxicity = model.predict_dataframe(data)

In [58]:
toxicity

array([0.87674659, 0.89244539, 0.01551677, ..., 0.86876005, 0.00147626,
       0.00403372])

In [59]:
toxicity.shape

(6698,)

In [60]:
# toxicity = (toxicity >= 0.5).astype(int)

In [61]:
toxicity

array([0.87674659, 0.89244539, 0.01551677, ..., 0.86876005, 0.00147626,
       0.00403372])

In [62]:
predict_label = toxicity.tolist()


In [63]:
data['predict_label'] = predict_label

In [64]:
data

,index,text,label,predict_label
0,0,Giờ biết giải thích sao với lão tôn đây 😅,1,0.876747
1,1,Nguyễn Đức Duẩn Thanh niên nhà Hà Nội thích N ...,0,0.892445
2,2,Khỉ bố đang rất đau khổ,0,0.015517
3,3,Hàng xóm bên cạnh ngta cũng phận nữ nhi mà các...,1,0.692576
4,4,"Sinh nở đã vất vả lắm rồi, đừng hỏi bố đứa trẻ...",1,0.154436
...,...,...,...,...
6693,6693,"Khổ nhưng vẫn đc độc lập tự do,được đón lễ lớn...",0,0.725702
6694,6694,30/4 3 que đu càng tụt quần bay trong gió😂,0,0.359476
6695,6695,ba que khát nước,0,0.868760
6696,6696,Hoan hô Việt Tân đưa tin rất hay và thời sự,0,0.001476


In [65]:
data.to_csv('expand_dataset/model_predict.csv', index=False)

In [33]:
similar_label = data.loc[data['label'] == data['predict_label']]

In [44]:
mislabel = data.loc[data['label'] != data['predict_label']]
mislabel.reset_index(drop=True, inplace=True)

In [39]:
similar_label.reset_index(drop=True, inplace=True)

In [40]:
len(similar_label)

4079

In [41]:
def view_data(df, start, view_range):
    for i in range(start, start+ view_range):
        print(f"Text: {df['text'][i]}")
        print(f"Label: {df['label'][i]}")
        print(f"Predict Label: {df['predict_label'][i]}")
        print('-------------------------------------')
        print("\n")

In [46]:
view_data(mislabel, 500, 20)

Text: Ăn vả ý :))
Label: 1
Predict Label: 0
-------------------------------------


Text: Mấy món này nó lại đơn giản luôn :)))
Label: 1
Predict Label: 0
-------------------------------------


Text: Nó cho ăn cái đập bạn ạ😁😁😁
Label: 1
Predict Label: 0
-------------------------------------


Text: Chủ yếu toàn thịt không. Không mê lắm 😂😂😂😂😂
Label: 1
Predict Label: 0
-------------------------------------


Text: Do nhà mình ăn ít thịt hay do nhà ng ta ăn nhiều thịt nhỉ
Label: 1
Predict Label: 0
-------------------------------------


Text: Quyền thực đơn đây, đừng hỏi e muốn ăn gì nữa nhé :))
Label: 1
Predict Label: 0
-------------------------------------


Text: Ăn quả mướp đắng à
Label: 1
Predict Label: 0
-------------------------------------


Text: Lê Tùng lưu vào nấu cho vk ăn nha 😂
Label: 0
Predict Label: 1
-------------------------------------


Text: Đoàn Thế Anh
Label: 1
Predict Label: 0
-------------------------------------


Text: Dương Bích ước
Label: 1
Predict Label: 0
----